# IEOR 4703 -- Monte Carlo Simulation for FE

## American option pricing via simulation

In [ ]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
# import our polynomial function
from polynomials import constructX
# linear regression
from sklearn.linear_model import LinearRegression

### Parameters

In [ ]:
r = 0.10
q = 0.01
s0 = 100
K = 100
sig = 0.40
T = 1

m = 365 # daily excercise
numPaths = 10000

dt = T/m
indicators = np.zeros(numPaths)

### Algorithm

In [ ]:
sMin = 10
sMax = 400

# number of basis
d = 4

typeOfPolynomial = 'Laguerre'

st = time()

#Initiate paths (not efficient; we should utilize the Brownian Bridge)
# simulate stock prices
s = np.zeros((m+1, numPaths))
t = np.linspace(0, T, m+1)
z = np.random.randn(m, numPaths)
delta_log_s = (r - q - sig*sig/2)*dt + sig*np.sqrt(dt)*z
log_s = np.zeros((m+1, numPaths))
log_s[0,:] = np.log(s0)
log_s[1:,:] = delta_log_s
log_s = np.cumsum(log_s, 0)
s = np.exp(log_s)

P = np.maximum(K - s[m,:], 0)
#print(np.exp(-r*T)*np.mean(P))
vHat = np.zeros(numPaths)

for i in np.arange(m-1, 0, -1):
    s_i = s[i, :]
    g = np.maximum(K - s_i, 0)
    # in-the-money indicator
    indicator = (g > 0)
    xi = s_i[indicator]
    if typeOfPolynomial == 'Laguerre':
        xXi = constructX(xi, typeOfPolynomial, d)
    elif typeOfPolynomial == 'Chebychev_firstKind' or typeOfPolynomial == 'Chebychev_secondKind':
        # make it shifted from (-1,1) to (sMin, sMax)
        xXi = constructX(2*(xi - sMin)/(sMax - sMin) - 1, typeOfPolynomial, d)
    yi = np.exp(-r*dt)*P[indicator]
    # regression
    reg = LinearRegression(fit_intercept=False).fit(xXi, yi)
    alpha = reg.coef_
    vH = np.dot(xXi, alpha)
    vHat[indicator] = vH

    indx = np.logical_and(indicator, g > vHat)
    P[indx] = g[indx]
    P[~indx] = np.exp(-r*dt)*P[~indx]

et = time()

print('Number of sims: %i' % numPaths)
print('Elapsed times was %f seconds' % (et-st))
premiumHat = np.exp(-r*dt) * np.mean(P)
print('American price: %f' % premiumHat)

Number of sims: 10000
Elapsed times was 0.896701 seconds
American price: 12.286710
